In [1]:
import pandas as pd
import numpy as np
import joblib
from datetime import datetime

# For modeling
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier


In [2]:
# Patients table: includes Id (unique), demographics, etc.
patients = pd.read_csv("patients.csv")

print("Patients shape:", patients.shape)
print(patients.head())


Patients shape: (22818, 28)
                                     Id   BIRTHDATE   DEATHDATE          SSN  \
0  3677432a-971b-7ef0-acbb-43ce84a150cc  2020-01-01         NaN  999-60-9177   
1  3e635a70-e4f8-0dab-0cea-e241e603d1c6  2008-10-13         NaN  999-76-7313   
2  146b2833-24a9-8087-70b2-f73790020184  1981-08-08  2010-12-22  999-67-8888   
3  7b00b9f8-8b8c-1338-4219-7c291cb09495  1996-04-24         NaN  999-64-6927   
4  bbb163df-264a-71aa-e4d8-091553d95dd1  1982-05-04         NaN  999-79-2179   

     DRIVERS    PASSPORT PREFIX       FIRST       MIDDLE        LAST  ...  \
0        NaN         NaN    NaN   Carlos172  Gregorio366  Estévez304  ...   
1  S99990286         NaN    NaN    Cathi439      Sana377      Fay398  ...   
2  S99969795  X68345241X    Mr.  Nickolas58       Jed345    Adams676  ...   
3  S99911456  X46352131X    Mr.    Felipe97          NaN    Muñoz828  ...   
4  S99950614  X75923971X   Mrs.     Ping698   Natasha268   Jacobi462  ...   

           CITY       STATE 

In [3]:
# Conditions table: includes patient diagnoses (CODE, DESCRIPTION, PATIENT)
conditions = pd.read_csv("conditions.csv")

print("Conditions shape:", conditions.shape)
print(conditions.head())


Conditions shape: (872128, 7)
        START        STOP                               PATIENT  \
0  2020-01-01  2020-02-05  3677432a-971b-7ef0-acbb-43ce84a150cc   
1  2020-04-08  2020-06-10  3677432a-971b-7ef0-acbb-43ce84a150cc   
2  2020-09-09  2020-12-09  3677432a-971b-7ef0-acbb-43ce84a150cc   
3  2020-10-30  2020-11-10  3677432a-971b-7ef0-acbb-43ce84a150cc   
4  2021-03-10  2021-03-10  3677432a-971b-7ef0-acbb-43ce84a150cc   

                              ENCOUNTER     SYSTEM       CODE  \
0  f61d3b3c-812c-cd92-e0e8-a0d5822e2c3b  SNOMED-CT  314529007   
1  d51fe3e7-1810-7c3f-db05-79de0ae075f8  SNOMED-CT  314529007   
2  510c7ae6-0658-21fe-7fa8-d43b9e56d16d  SNOMED-CT  314529007   
3  f56f3a8d-6440-b474-ba3e-47d073175dad  SNOMED-CT  195662009   
4  dfb1b143-9219-8cc2-b46a-7bc95c7b7d8a  SNOMED-CT  314529007   

                          DESCRIPTION  
0   Medication review due (situation)  
1   Medication review due (situation)  
2   Medication review due (situation)  
3  Acute viral p

In [4]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
observations_file = "/content/drive/MyDrive/observations.csv"


In [6]:
import pandas as pd

chunksize = 100_000
obs_chunks = []

for chunk in pd.read_csv(observations_file, chunksize=chunksize):
    obs_chunks.append(chunk)

observations = pd.concat(obs_chunks, ignore_index=True)

print("Observations shape:", observations.shape)
print(observations.head())


Observations shape: (19161370, 9)
                   DATE                               PATIENT  \
0  2020-01-01T07:00:09Z  3677432a-971b-7ef0-acbb-43ce84a150cc   
1  2020-01-01T07:00:09Z  3677432a-971b-7ef0-acbb-43ce84a150cc   
2  2020-01-01T07:00:09Z  3677432a-971b-7ef0-acbb-43ce84a150cc   
3  2020-01-01T07:00:09Z  3677432a-971b-7ef0-acbb-43ce84a150cc   
4  2020-01-01T07:00:09Z  3677432a-971b-7ef0-acbb-43ce84a150cc   

                              ENCOUNTER     CATEGORY     CODE  \
0  f61d3b3c-812c-cd92-e0e8-a0d5822e2c3b  vital-signs   8302-2   
1  f61d3b3c-812c-cd92-e0e8-a0d5822e2c3b  vital-signs  72514-3   
2  f61d3b3c-812c-cd92-e0e8-a0d5822e2c3b  vital-signs  29463-7   
3  f61d3b3c-812c-cd92-e0e8-a0d5822e2c3b  vital-signs  77606-2   
4  f61d3b3c-812c-cd92-e0e8-a0d5822e2c3b  vital-signs   8289-1   

                                         DESCRIPTION VALUE    UNITS     TYPE  
0                                        Body Height  51.0       cm  numeric  
1  Pain severity - 0-10 ve

In [15]:
# Step 1: define T2DM codes
t2dm_codes = [
    "44054006",    # T2DM
]

# Step 2: filter conditions for T2DM
# t2dm_rows = conditions[conditions["CODE"].isin(t2dm_codes)]
# Step 2: filter with CODE as string
t2dm_rows = conditions[conditions["CODE"].astype(str).isin(t2dm_codes)]


# Step 3: build a set of T2DM patient IDs
t2dm_patient_ids = set(t2dm_rows["PATIENT"])
print("Total T2DM patients:", len(t2dm_patient_ids))

# Step 4: label each patient
patients["diabetes_label"] = patients["Id"].apply(
    lambda pid: 1 if pid in t2dm_patient_ids else 0
)
print("Label distribution:\n", patients["diabetes_label"].value_counts())


Total T2DM patients: 1434
Label distribution:
 diabetes_label
0    21384
1     1434
Name: count, dtype: int64


In [16]:
feature_map = {
    "Hemoglobin A1c/Hemoglobin.total in Blood": "hba1c",
    "Glucose [Mass/volume] in Blood": "glucose",
    "Body mass index (BMI) [Ratio]": "bmi",
    "Body Weight": "weight",
    "Body Height": "height",
    "Systolic Blood Pressure": "systolic_bp",
    "Diastolic Blood Pressure": "diastolic_bp",
    "Cholesterol [Mass/volume] in Serum or Plasma": "cholesterol",
    "Low Density Lipoprotein Cholesterol": "ldl",
    "Glomerular filtration rate/1.73 sq M.predicted [Volume Rate/Area] in Serum or Plasma by Creatinine-based formula (MDRD)": "egfr"
}


In [17]:
# Filter
obs_filtered = observations[observations["DESCRIPTION"].isin(feature_map.keys())].copy()

# Rename columns to a new "feature" column
obs_filtered["feature"] = obs_filtered["DESCRIPTION"].map(feature_map)

# Convert VALUE to numeric
obs_filtered["VALUE"] = pd.to_numeric(obs_filtered["VALUE"], errors="coerce")

# Convert DATE to datetime
obs_filtered["DATE"] = pd.to_datetime(obs_filtered["DATE"], errors="coerce")

# We'll take the LATEST value per patient-feature
obs_filtered.sort_values("DATE", inplace=True)
latest_obs = obs_filtered.groupby(["PATIENT", "feature"], as_index=False).last()

# Pivot to wide form: one row per patient, columns = features
patient_features = latest_obs.pivot(index="PATIENT", columns="feature", values="VALUE").reset_index()
patient_features.rename(columns={"PATIENT": "Id"}, inplace=True)

print("Patient features shape:", patient_features.shape)
print(patient_features.head())


Patient features shape: (22818, 11)
feature                                    Id   bmi  cholesterol  \
0        0000ce8b-d02f-56b8-92e7-e7a027f0b75c  24.5          NaN   
1        0000f379-6d8a-044d-5922-a9635d745741  30.6        277.2   
2        00012b04-c9a9-38b6-3880-299d710a71da  30.8        189.2   
3        000144ed-006e-e7e7-6ae2-f02b5085be1b  23.7          NaN   
4        00020de6-93ac-4937-b1a0-49c577074b5f  20.8          NaN   

feature  diastolic_bp  egfr  glucose  hba1c  height    ldl  systolic_bp  \
0                82.7   NaN      NaN    NaN   181.6    NaN         81.2   
1                59.0  10.7      NaN    5.6   170.3  199.2         96.0   
2                63.0   NaN      NaN    NaN   171.6  106.4         83.0   
3                74.0  11.2      NaN    NaN   148.8    NaN        111.0   
4                83.0   NaN      NaN    NaN    89.9    NaN        130.0   

feature  weight  
0          80.7  
1          88.8  
2          90.7  
3          52.5  
4          16.

In [18]:
final_df = pd.merge(patients, patient_features, on="Id", how="left")

print("After merge:", final_df.shape)
print(final_df.head())

# Example columns now:
# ["Id", ..., "diabetes_label", "glucose", "hba1c", "bmi", "ldl", "cholesterol", ...]


After merge: (22818, 39)
                                     Id   BIRTHDATE   DEATHDATE          SSN  \
0  3677432a-971b-7ef0-acbb-43ce84a150cc  2020-01-01         NaN  999-60-9177   
1  3e635a70-e4f8-0dab-0cea-e241e603d1c6  2008-10-13         NaN  999-76-7313   
2  146b2833-24a9-8087-70b2-f73790020184  1981-08-08  2010-12-22  999-67-8888   
3  7b00b9f8-8b8c-1338-4219-7c291cb09495  1996-04-24         NaN  999-64-6927   
4  bbb163df-264a-71aa-e4d8-091553d95dd1  1982-05-04         NaN  999-79-2179   

     DRIVERS    PASSPORT PREFIX       FIRST       MIDDLE        LAST  ...  \
0        NaN         NaN    NaN   Carlos172  Gregorio366  Estévez304  ...   
1  S99990286         NaN    NaN    Cathi439      Sana377      Fay398  ...   
2  S99969795  X68345241X    Mr.  Nickolas58       Jed345    Adams676  ...   
3  S99911456  X46352131X    Mr.    Felipe97          NaN    Muñoz828  ...   
4  S99950614  X75923971X   Mrs.     Ping698   Natasha268   Jacobi462  ...   

    bmi cholesterol diastolic_b

In [19]:
final_df["AGE"] = pd.to_datetime(final_df["BIRTHDATE"], errors="coerce").apply(
    lambda dob: datetime.now().year - dob.year if pd.notnull(dob) else np.nan
)


In [20]:
columns_to_keep = [
    "Id", "diabetes_label", "AGE",
    "glucose", "hba1c", "bmi", "ldl", "cholesterol",
    "systolic_bp", "diastolic_bp", "egfr", "weight", "height"
]
final_df_cleaned = final_df[columns_to_keep]
print("Final columns:", final_df_cleaned.columns.tolist())
print(final_df_cleaned.head())


Final columns: ['Id', 'diabetes_label', 'AGE', 'glucose', 'hba1c', 'bmi', 'ldl', 'cholesterol', 'systolic_bp', 'diastolic_bp', 'egfr', 'weight', 'height']
                                     Id  diabetes_label  AGE  glucose  hba1c  \
0  3677432a-971b-7ef0-acbb-43ce84a150cc               0    5      NaN    NaN   
1  3e635a70-e4f8-0dab-0cea-e241e603d1c6               0   17      NaN    NaN   
2  146b2833-24a9-8087-70b2-f73790020184               0   44     89.5    5.4   
3  7b00b9f8-8b8c-1338-4219-7c291cb09495               0   29      NaN    NaN   
4  bbb163df-264a-71aa-e4d8-091553d95dd1               0   43      NaN    NaN   

    bmi    ldl  cholesterol  systolic_bp  diastolic_bp   egfr  weight  height  
0  15.4    NaN          NaN        132.0          80.0    NaN    17.1   105.5  
1  23.2    NaN          NaN        127.0          68.0    NaN    59.3   159.9  
2  29.2   77.6        156.7        101.0          57.0  105.4    91.5   177.0  
3  28.6    NaN          NaN        114.0    

In [21]:
# We'll fill numeric columns with mean
lab_cols = [
    "glucose", "hba1c", "bmi", "ldl", "cholesterol",
    "systolic_bp", "diastolic_bp", "egfr", "weight", "height"
]
final_df_cleaned[lab_cols] = final_df_cleaned[lab_cols].fillna(final_df_cleaned[lab_cols].mean())


<ipython-input-21-b9e3b0a81082>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df_cleaned[lab_cols] = final_df_cleaned[lab_cols].fillna(final_df_cleaned[lab_cols].mean())


In [22]:
X = final_df_cleaned[lab_cols]
y = final_df_cleaned["diabetes_label"]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

model = RandomForestClassifier(
    n_estimators=100,
    class_weight="balanced_subsample",  # helps handle imbalance
    random_state=42
)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.98      1.00      0.99      4277
           1       0.96      0.75      0.84       287

    accuracy                           0.98      4564
   macro avg       0.97      0.87      0.91      4564
weighted avg       0.98      0.98      0.98      4564

